<a href="https://colab.research.google.com/github/cat-on-tree/HuggingFace-Tutorials/blob/main/6_BERT_%E4%B8%AD%E6%96%87%E5%A1%AB%E7%A9%BA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 13.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system =

In [2]:
import torch
import torch.nn as nn
from datasets import load_dataset
from transformers import BertTokenizer
from transformers import BertModel
from torch.optim import AdamW
import random
import numpy as np

In [3]:
# 设置随机种子
seed = 42
torch.manual_seed(seed)
random.seed(seed)
np.random.seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

In [4]:
#检查GPU可用性，使用GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [5]:
#查看数据集构成
dataset_view = load_dataset(path='lansinuote/ChnSentiCorp')
dataset_view

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


dataset_infos.json:   0%|          | 0.00/960 [00:00<?, ?B/s]

(…)-00000-of-00001-02f200ca5f2a7868.parquet:   0%|          | 0.00/2.16M [00:00<?, ?B/s]

(…)-00000-of-00001-405befbaa3bcf1a2.parquet:   0%|          | 0.00/276k [00:00<?, ?B/s]

(…)-00000-of-00001-5372924f059fe767.parquet:   0%|          | 0.00/275k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/9600 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1200 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 9600
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 1200
    })
})

In [6]:
#定义数据集
class Dataset(torch.utils.data.Dataset):
    def __init__(self, split):
        self.dataset = load_dataset(path='lansinuote/ChnSentiCorp', split=split)
#不筛选掉小于30个token长度的样本
        # def f(data):
        #     return len(data['text']) > 30

        # self.dataset = dataset.filter(f)

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, i):
        text = self.dataset[i]['text']

        return text

In [7]:
#加载字典和分词工具
token = BertTokenizer.from_pretrained('bert-base-chinese')

token

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

BertTokenizer(name_or_path='bert-base-chinese', vocab_size=21128, model_max_length=512, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
)

In [8]:
def collate_fn(data):
    # 编码
    data = token.batch_encode_plus(batch_text_or_text_pairs=data,
                                   truncation=True,
                                   padding=True,  # 动态填充
                                   return_tensors='pt',
                                   return_length=True)

    # input_ids: 编码之后的数字
    # attention_mask: 补零的位置是0, 其他位置是1
    input_ids = data['input_ids']
    attention_mask = data['attention_mask']
    token_type_ids = data['token_type_ids']

    # input_ids 是形状为 (batch_size, seq_length) 的张量
    batch_size, seq_length = input_ids.shape

    # 随机选择一个有效词的位置进行 mask
    valid_positions = attention_mask.nonzero(as_tuple=True)  # 获取有效位置
    mask_positions = []
    for i in range(batch_size):
        positions = valid_positions[1][valid_positions[0] == i]
        mask_positions.append(random.choice(positions.tolist()))
    mask_positions = torch.tensor(mask_positions)

    # 生成 labels
    labels = input_ids[torch.arange(batch_size), mask_positions].clone()

    # 替换为 mask token
    input_ids[torch.arange(batch_size), mask_positions] = token.get_vocab()[token.mask_token]
    #将数据转移到 GPU
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    token_type_ids = token_type_ids.to(device)
    labels = labels.to(device)
    mask_positions = mask_positions.to(device)

    # 返回数据
    return input_ids, attention_mask, token_type_ids, labels, mask_positions

In [9]:
#数据加载器
train_loader = torch.utils.data.DataLoader(dataset=Dataset('train'),
                                     batch_size=16,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

val_loader = torch.utils.data.DataLoader(dataset=Dataset('validation'),
                                     batch_size=16,
                                     collate_fn=collate_fn,
                                     shuffle=True,
                                     drop_last=True)

for i, (input_ids, attention_mask, token_type_ids, labels, mask_positions) in enumerate(train_loader):
    break

for i, (input_ids, attention_mask, token_type_ids, labels, mask_positions) in enumerate(val_loader):
    break

print(len(train_loader))
print(len(val_loader))
print(token.decode(input_ids[0]))
print(token.decode(labels[0]))
input_ids.shape, attention_mask.shape, token_type_ids.shape, labels.shape

600
75
[CLS] 非 常 好 的 一 本 书 ！ 因 为 学 校 搞 图 书 交 流 活 动 ， 儿 子 向 别 人 换 了 这 本 书 ， 可 惜 还 没 看 完 ， 别 人 就 催 着 要 还 ， 孩 子 很 舍 不 得 ， 可 是 别 人 催 得 很 紧 ， 孩 子 跟 我 说 ， 我 把 它 抄 下 来 吧 ！ 很 喜 欢 的 样 子 ， 我 就 到 当 当 上 订 了 ， 送 货 很 快 ， 质 量 也 不 错 ， 最 主 要 的 是 孩 子 喜 欢 得 不 得 了 ！ 我 一 晚 上 看 完 了 ， 的 确 不 [MASK] ， 看 了 后 回 忆 起 自 己 的 一 年 级 ， 好 像 也 是 这 样 的 [UNK] [UNK] 只 是 在 想 作 者 为 什 么 不 出 一 本 二 年 级 的 ， 孩 子 马 上 上 二 年 级 了 ， 接 着 写 呀 ^ - ^ 期 待 中 [UNK] [UNK] [SEP]
错


(torch.Size([16, 194]),
 torch.Size([16, 194]),
 torch.Size([16, 194]),
 torch.Size([16]))

In [10]:
#加载预训练模型
pretrained = BertModel.from_pretrained('bert-base-chinese')
pretrained.to(device)
#不对预训练模型微调，无需计算梯度
for param in pretrained.parameters():
    param.requires_grad_(False)
#模型试算
check = pretrained(input_ids=input_ids,
           attention_mask=attention_mask,
           token_type_ids=token_type_ids)

check.last_hidden_state.shape

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

torch.Size([16, 194, 768])

In [11]:
#定义下游任务模型
class Model(nn.Module):
    def __init__(self, pretrained, token):
        super().__init__()
        self.pretrained = pretrained  # 预训练模型
        hidden_size = self.pretrained.config.hidden_size  # 动态获取隐藏层维度
        self.decoder = nn.Linear(hidden_size, token.vocab_size, bias=False)
        self.bias = nn.Parameter(torch.zeros(token.vocab_size))  # 解码器的偏置
        self.decoder.bias = self.bias

    def forward(self, input_ids, attention_mask, token_type_ids, mask_positions):
        # 使用预训练模型提取特征
        out = self.pretrained(input_ids=input_ids,
                              attention_mask=attention_mask,
                              token_type_ids=token_type_ids)

        # 提取 mask_positions 对应的隐藏状态
        batch_size = input_ids.size(0)
        if batch_size == 0:  # 检查批次大小
            return None

        hidden_states = out.last_hidden_state  # 形状: (batch_size, seq_length, hidden_size)
        assert hidden_states.size(1) == input_ids.size(1), "Sequence length mismatch"
        assert hidden_states.size(2) == self.decoder.in_features, "Hidden size mismatch"

        # 根据 mask_positions 提取隐藏状态
        selected_hidden_states = hidden_states[torch.arange(batch_size), mask_positions]  # 形状: (batch_size, hidden_size)

        # 解码到词表维度
        out = self.decoder(selected_hidden_states)  # 输出形状: (batch_size, vocab_size)
        return out

# 实例化模型并移动到 GPU
model = Model(pretrained=pretrained, token=token).to(device)

# 调用模型
model(input_ids=input_ids,
               attention_mask=attention_mask,
               token_type_ids=token_type_ids,
               mask_positions=mask_positions)

tensor([[-0.7683, -0.3043, -1.1173,  ..., -0.1415, -0.5495, -0.2773],
        [-0.3929, -0.6316, -0.5033,  ..., -0.1709, -0.2723, -0.6200],
        [ 0.0029, -0.4255, -0.2277,  ..., -0.1636, -0.3296, -0.0851],
        ...,
        [-0.1493, -0.3952, -0.5897,  ...,  0.4546, -0.9737,  0.1490],
        [-0.5897, -0.5682, -0.6041,  ..., -0.6446, -0.4130,  0.1753],
        [-0.3236, -0.0470, -0.1054,  ..., -0.1721, -0.6965, -0.1927]],
       device='cuda:0', grad_fn=<AddmmBackward0>)

In [13]:
# 定义损失函数和优化器
criterion = torch.nn.CrossEntropyLoss()  # 损失函数
optimizer = AdamW(model.parameters(), lr=5e-4)  # 优化器

# 定义训练参数
num_epochs = 3  # 总训练轮数
loss_threshold = 1e-4  # 损失变化阈值（早停条件）
prev_loss = float('inf')  # 初始化上一个 epoch 的 loss

for epoch in range(num_epochs):  # 外层循环负责 epoch
    model.train()  # 确保模型处于训练模式
    print(f"Epoch {epoch + 1}/{num_epochs}")

    epoch_loss = 0
    total_samples = 0  # 用于精确计算平均 loss

    for i, (input_ids, attention_mask, token_type_ids, labels, mask_positions) in enumerate(train_loader):
        # 前向传播
        out = model(input_ids=input_ids,
                    attention_mask=attention_mask,
                    token_type_ids=token_type_ids,
                    mask_positions=mask_positions)

        # 计算损失
        loss = criterion(out, labels)

        # 反向传播
        optimizer.zero_grad()  # 清空梯度
        loss.backward()  # 反向传播
        optimizer.step()  # 更新参数

        # 累加 epoch 的 loss
        batch_size = input_ids.size(0)
        epoch_loss += loss.item() * batch_size  # 按样本权重累加
        total_samples += batch_size

        # 打印日志信息（每隔 50 个 batch 打印一次）
        if i % 50 == 0:
            out = out.argmax(dim=1)
            accuracy = (out == labels).sum().item() / len(labels)
            print(f"Batch {i}, Loss: {loss.item():.4f}, Accuracy: {accuracy:.4f}")

    # 计算 epoch 的平均 loss
    epoch_loss /= total_samples
    print(f"Epoch {epoch + 1}, Average Loss: {epoch_loss:.4f}")

    # 检查 Loss 是否变化不明显（早停机制）
    if abs(prev_loss - epoch_loss) < loss_threshold:
        print("Training loss converged. Stopping early.")
        break
    prev_loss = epoch_loss

Epoch 1/3
Batch 0, Loss: 5.0375, Accuracy: 0.1875
Batch 50, Loss: 5.6890, Accuracy: 0.3750
Batch 100, Loss: 6.4444, Accuracy: 0.1875
Batch 150, Loss: 4.7124, Accuracy: 0.4375
Batch 200, Loss: 5.1729, Accuracy: 0.3125
Batch 250, Loss: 6.0342, Accuracy: 0.3125
Batch 300, Loss: 3.2900, Accuracy: 0.6250
Batch 350, Loss: 3.6205, Accuracy: 0.5000
Batch 400, Loss: 2.3689, Accuracy: 0.6250
Batch 450, Loss: 2.9378, Accuracy: 0.4375
Batch 500, Loss: 2.3814, Accuracy: 0.7500
Batch 550, Loss: 3.3048, Accuracy: 0.5625
Epoch 1, Average Loss: 4.6189
Epoch 2/3
Batch 0, Loss: 3.0766, Accuracy: 0.5625
Batch 50, Loss: 3.5646, Accuracy: 0.4375
Batch 100, Loss: 2.4106, Accuracy: 0.7500
Batch 150, Loss: 2.9478, Accuracy: 0.5625
Batch 200, Loss: 2.4374, Accuracy: 0.6875
Batch 250, Loss: 3.1520, Accuracy: 0.3750
Batch 300, Loss: 2.6727, Accuracy: 0.5625
Batch 350, Loss: 2.3592, Accuracy: 0.4375
Batch 400, Loss: 3.8420, Accuracy: 0.5625
Batch 450, Loss: 1.7079, Accuracy: 0.8125
Batch 500, Loss: 3.0168, Accurac

In [14]:
# 测试函数
def test():
    model.eval()  # 设置模型为评估模式
    correct = 0
    total = 0

    # 测试集数据加载器
    loader_test = torch.utils.data.DataLoader(dataset=Dataset('test'),
                                              batch_size=32,
                                              collate_fn=collate_fn,
                                              shuffle=False,  # 测试集通常不需要打乱
                                              drop_last=False)  # 测试集不需要丢弃最后一个 batch

    # 遍历测试集
    for i, (input_ids, attention_mask, token_type_ids, labels, mask_positions) in enumerate(loader_test):

        # 禁用梯度计算（测试阶段无需反向传播）
        with torch.no_grad():
            out = model(input_ids=input_ids,
                        attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        mask_positions=mask_positions)

        # 获取预测值
        out = out.argmax(dim=1)

        # 更新正确预测的数量和总样本数
        correct += (out == labels).sum().item()
        total += len(labels)

        # 打印日志信息（仅打印前 5 个样本）
        if i < 5:
            print(f"Batch {i}:")
            print(f"Decoded Input: {token.decode(input_ids[0].tolist())}")
            print(f"True Label: {token.decode([labels[0].item()])}")
            print(f"Predicted Label: {token.decode([out[0].item()])}")
            print("-" * 50)

    # 打印测试准确率
    accuracy = correct / total
    print(f"Test Accuracy: {accuracy:.4f}")


# 调用测试函数
test()

Batch 0:
Decoded Input: [CLS] 这 个 宾 馆 比 较 陈 旧 了 ， 特 价 的 房 间 也 很 一 般 。 总 体 来 说 [MASK] 般 [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [